## EDA Analysis
Table iar_ocupaciones

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

In [ ]:
def millions(x, pos):
    return f'{x * 1e-6:.1f}M'  # Convert to millions and format with one decimal place


def graph(x1, y1, ylabel1, y2, ylabel2):
    fig, ax = plt.subplots(2, 1, figsize=(24, 10), sharex=True)

    ax[0].plot(x1, y1, color='blue', label=ylabel1, marker='o', linestyle='-', linewidth=2)
    ax[0].set_title(f'{ylabel1} Over Time')
    ax[0].set_xlabel("Date")
    ax[0].set_ylabel(f'{ylabel1} (in Millions)')
    ax[0].yaxis.set_major_formatter(FuncFormatter(millions))  # Apply the formatter to the y-axis
    ax[0].tick_params(axis='x', which='both', labelbottom=True)
    ax[0].legend()

    ax[1].plot(x1, y2, color='red', label=ylabel2, marker='o', linestyle='-', linewidth=2)
    ax[1].set_title(f'{ylabel2} Over Time')
    ax[1].set_xlabel("Date")
    ax[1].set_ylabel(f'{ylabel2} (in Millions)')
    ax[1].yaxis.set_major_formatter(FuncFormatter(millions))  # Apply the formatter to the y-axis
    ax[1].tick_params(axis='x', which='both', labelbottom=True)
    ax[1].legend()

    # Add grid lines
    for axis in ax:
        axis.tick_params(axis='x', rotation=45)  # Rotate x-axis labels for readability
        axis.grid(True, linestyle='--', alpha=0.5)

    # Adjust layout
    plt.tight_layout()

    # Show the plot
    plt.show()

def histogram(column, label, threshold=0):
    plt.figure(figsize=(24, 6))
    columna = column.value_counts()

    # less_than = columna[columna < threshold].sum()
    main = columna[columna >= threshold]

    # pd.concat([main, pd.Series({'Other': less_than})]).plot(kind='bar', color='skyblue', edgecolor='black')
    main.plot(kind='bar', color='skyblue', edgecolor='black')

    # Add title and labels
    plt.title(f'Occurrences of Each Factor in {label}')
    plt.xlabel(label)
    plt.ylabel('Count')

    # Show the plot
    plt.show()

In [ ]:
df2 = pd.read_csv("iar_ocupaciones.csv")

In [ ]:
df2.info()

In [ ]:
df = df2.drop_duplicates()
df['Fecha_hoy'] = pd.to_datetime(df['Fecha_hoy'])

In [ ]:
print(len(df2)) 
print(len(df))

In [ ]:
abs(len(df2) - len(df))

In [ ]:
df.columns

### Group By Date

In [ ]:
df.head()

In [ ]:
# Dataframe for numerical columns and date
df_num = df[['Fecha_hoy', 'ing_hab', 'aa_ing_hab',
             'ing_hab_nocoruso', 'aa_ing_hab_nocoruso']]
df_num['monthly'] = df['Fecha_hoy'].dt.strftime('%Y-%m')
df_num = df_num.drop("Fecha_hoy", axis=1)
df_num.head()

#### ing_hab and aa_ing_hab over time

In [ ]:
df_num = df_num.groupby(by="monthly").sum()

In [ ]:
graph(df_num.index, df_num["ing_hab"], "ing_hab", df_num["aa_ing_hab"], "aa_ing_hab")


#### ing_hab_nocoruso and aa_ing_hab_nocoruso over time

In [ ]:
graph(df_num.index, df_num["ing_hab_nocoruso"], "ing_hab_nocoruso", df_num["aa_ing_hab_nocoruso"], "aa_ing_hab_nocoruso")

### Histograms

In [ ]:
histogram(df["ID_Tipo_Habitacion"], "ID_Tipo_Habitacion", 1000)

In [ ]:
histogram(df["ID_Agencia"], "ID_Agencia", 2000)

In [ ]:
histogram(df["ID_Entidad_Fed"], "ID_Entidad_Fed", 2000)

In [ ]:
histogram(df["ID_canal"], "ID_canal")

In [ ]:
histogram(df["ID_Programa"], "ID_Programa")

In [ ]:
histogram(df["ID_Paquete"], "ID_Paquete")

In [ ]:
histogram(df["ID_Segmento_Mercado"], "ID_Segmento_Mercado")

In [ ]:
histogram(df["ID_Segmento_Alterno"], "ID_Segmento_Alterno")

In [ ]:
histogram(df["ID_Segmento_Comp"], "ID_Segmento_Comp")

In [ ]:
histogram(df["ID_empresa"], "ID_empresa")

In [ ]:
histogram(df["ID_Empresa2"], "ID_Empresa2")

#### Matrices de Correlación

In [ ]:
df_original = df2.copy()
df_sample = df_original.sample(frac=0.05, random_state=52)  # Adjust the fraction as needed
df_sample = df_sample.loc[:, df_sample.sum() != 0] # Remove columns with only 0s

In [ ]:
# Separamos los ID's del df
df_id = df_sample.filter(regex="ID")
df_non_id = df_sample.loc[:, ~df_sample.columns.str.contains("ID|aa")]

In [ ]:
# Correlation Matrix ID's
plt.figure(figsize=(14, 8))
sns.heatmap(df_id.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.show()

In [ ]:
# Correlation Matrix for non ID
plt.figure(figsize=(14, 8))
sns.heatmap(df_non_id.iloc[:,1:].corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.show()

#### Boxplots

In [ ]:
for column in df_non_id.columns:
    if column == 'Fecha_hoy':
        continue
    plt.figure(figsize=(15, 2))
    df_non_id.boxplot(column=[column], vert=False)
    plt.title(f'Box Plot of {column}')
    plt.ylabel('Values')
    plt.show()